# Learning by Necessity Interactive

Original Matlab Code: Ethan Ilzetzki


In [1]:
import numpy as np
from numpy import inf
import pandas as pd

## Put everything together

In [2]:
R = 1.03 # real interest rate 
z = 0.3 
alph = 2/3 # typical labour share
d = 0.08 # depreciation rate
YPostWar = 0.3
YPreWar = 0.1 # 1938
YWar = [0.15, 0.325, 0.66, 1.2, 2.1, 2.4, 1.2, 0.9, 0.44] # 1939 to 1947 
PreWarLength = 5
WarLength = len(YWar)

phi = 1 # need to calibrate 
w = 0.526 
W = 0.24 * 0.25 * w
overtime = 0.5
FullTime = 0.24
delta = 0.0364
psi = 1 # need to calibrate

### Grids
T = 100
ngridLK = 50
ngridH = 10
grid = ngridLK**2 * ngridH

maxK = 20
maxL = 20
maxH = 0.35 
minH = 0.15

gridK = np.linspace(0, maxK, ngridLK)
gridL = np.linspace(0, maxL, ngridLK)
gridH = np.linspace(minH, maxH, ngridH)

### Matrices
HH_first = np.transpose(np.ones(ngridLK**2))
HH_second = np.kron(HH_first, np.transpose(gridH))
HH = np.tile(HH_second, (ngridLK**2,1)).transpose()

LLNext_first = np.kron(gridL.transpose(), np.ones(ngridH))
LLNext_second = np.kron(np.ones(ngridLK), LLNext_first)
LLNext = np.tile(LLNext_second, (ngridLK**2,1)).transpose()

KKNext_first = np.kron(gridK.transpose(), np.ones(ngridH * ngridLK))
KKNext = np.tile(KKNext_first, (ngridLK**2,1)).transpose()

LL_first = np.kron(np.ones(ngridLK), gridL)
LL = np.tile(LL_first, (grid,1))

KK_first = np.kron(gridK, np.ones(ngridLK))
KK = np.tile(KK_first, (grid,1))

Y = [[YPreWar], YWar, [YPostWar] * (T-WarLength-1)]
Y = list(np.concatenate(Y).flat) # flatten list

CostNPV = np.zeros((ngridLK**2, T+1))
Choices = np.zeros((ngridLK**2, T))
CostNow = np.zeros_like(HH)

### Cost Minimisation 
for t in range(T, 0, -1):
    step = t 
    
    UU_first = Y[t-1]/z
    UU_second = np.power((HH * LL), alph)
    UU_third = np.power(np.divide(UU_first, UU_second), 1/(1-alph))
    UU = np.divide(UU_third, KK)
    
    II = KKNext - np.multiply((1-d), KK)
    DD = LLNext - LL
    LaborCosts = W + w* HH + w*overtime*(HH-FullTime) * (HH > FullTime)
    
    CostNow_first = phi/2 * np.power(np.divide(II, KK) -d, 2) + (R-1) * KK
    CostNow_second = delta * np.divide(UU, (1-UU)) * KK
    CostNow_third = LaborCosts * (psi/2) * np.divide(np.power(DD, 2), LL)
    CostNow_fourth = LaborCosts * LL
    CostNow = CostNow_first + CostNow_second + CostNow_third + CostNow_fourth
    CostNow[UU>1] = inf
    
    CostNext_first = np.kron(CostNPV[:, t], np.ones(ngridH)) # be careful of whether it is t or t + 1
    CostNext = np.tile(CostNext_first, (ngridLK**2,1)).transpose()
    
    CostNPV[:,t-1] = np.amin((CostNow + np.divide(CostNext, R)), axis = 0) # column min 
    Choices[:, t-1] = np.argmin((CostNow + np.divide(CostNext, R)), axis = 0) 

/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in true_di

In [3]:
Choices

array([[    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [11610., 14180., 17780., ...,  7120., 11130., 24970.],
       [11610., 14180., 17780., ...,  7120., 11130., 24980.],
       [11610., 14180., 17780., ...,  7120., 11130., 24990.]])

In [4]:
pd.DataFrame(CostNPV)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,42.373190,42.578687,42.297026,41.433537,39.953535,37.934375,33.597115,28.373532,26.710211,26.276058,...,8.714542,8.136841,7.540840,6.926467,6.293323,5.640090,4.961914,4.196741,2.633684,0.0
2496,42.439493,42.642761,42.355850,41.479423,39.957822,37.939333,33.593269,28.397149,26.766149,26.341168,...,8.780121,8.202420,7.606691,6.992318,6.359175,5.705941,5.027765,4.262320,2.678561,0.0
2497,42.505853,42.706987,42.415054,41.526302,39.967643,37.906147,33.611477,28.425478,26.822888,26.406401,...,8.845792,8.268091,7.672623,7.058250,6.425106,5.771873,5.093697,4.327991,2.723451,0.0
2498,42.572266,42.771356,42.474615,41.574106,39.982213,37.893267,33.613057,28.457784,26.880349,26.471747,...,8.911548,8.333847,7.738629,7.124256,6.491113,5.837879,5.159703,4.393747,2.768354,0.0


In [5]:
KK

array([[ 0.,  0.,  0., ..., 20., 20., 20.],
       [ 0.,  0.,  0., ..., 20., 20., 20.],
       [ 0.,  0.,  0., ..., 20., 20., 20.],
       ...,
       [ 0.,  0.,  0., ..., 20., 20., 20.],
       [ 0.,  0.,  0., ..., 20., 20., 20.],
       [ 0.,  0.,  0., ..., 20., 20., 20.]])

In [20]:
UU

array([[       inf,        inf,        inf,        inf, 0.47407407,
        0.11851852,        inf, 0.23703704, 0.05925926],
       [       inf,        inf,        inf,        inf, 0.26666667,
        0.06666667,        inf, 0.13333333, 0.03333333],
       [       inf,        inf,        inf,        inf, 0.17066667,
        0.04266667,        inf, 0.08533333, 0.02133333],
       [       inf,        inf,        inf,        inf, 0.11851852,
        0.02962963,        inf, 0.05925926, 0.01481481],
       [       inf,        inf,        inf,        inf, 0.08707483,
        0.02176871,        inf, 0.04353741, 0.01088435],
       [       inf,        inf,        inf,        inf, 0.47407407,
        0.11851852,        inf, 0.23703704, 0.05925926],
       [       inf,        inf,        inf,        inf, 0.26666667,
        0.06666667,        inf, 0.13333333, 0.03333333],
       [       inf,        inf,        inf,        inf, 0.17066667,
        0.04266667,        inf, 0.08533333, 0.02133333],
